In [33]:
import numpy as np
import pandas as pd
from math import sqrt,exp,pi,log
from random import randrange

In [34]:
df1 = pd.read_csv(r"C:\Users\shruthi sree\Downloads\car.data",header = None)
df1.to_csv(r"C:\Users\shruthi sree\Desktop\ML_PA_1001933428\car.csv",header=["buying","maint","doors","persons","lug_boot","safety","performance"])
df1 = pd.read_csv(r"C:\Users\shruthi sree\Desktop\ML_PA_1001933428\car.csv")
data1 = df1.iloc[:,:].values

from sklearn.preprocessing import LabelEncoder  
lab = LabelEncoder()
for i in range(len(data1[0])):
    data1[:,i]=lab.fit_transform(data1[:,i])   
ulist = np.unique(data1[:,-1])


In [35]:
def eval_algo(data1, nfolds):
    folds = crossval_split(data1, nfolds)
    scores = list()
    i=0
    while i < (len(folds)):
        train_set=list()
        j=0
        while j< (len(folds)):
            if i!=j:
                train_set.append(folds[j])
            j+= 1
        train_set=list(folds)
        train_set = sum(train_set, [])
        test_set = list()
        for row in folds[i]:
            row1 = list(row)
            test_set.append(row1)
            row1[-1] = None
        pred = naive_bayes(train_set, test_set)
        act = [row[-1] for row in folds[i]]
        accuracy = acc_metric(act, pred)
        scores.append(accuracy)
        i+=1
    return scores

In [36]:
def crossval_split(data1, nfolds):
    data_split = list()
    data_copy = list(data1)
    fold_size = int(len(data1) / nfolds)
    for i in range(nfolds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(data_copy))
            fold.append(data_copy.pop(index))
        data_split.append(fold)
    return data_split

In [37]:
def naive_bayes(train, test):
    print(train)
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

In [38]:
def summarize_by_class(data1):
    separated = split_by_class(data1,ulist)
    summaries = dict()
    for value, rows in separated.items():
        summaries[value] = summarize(rows)
    return summaries

In [39]:
def split_by_class(data1,ulist):
    slist=dict()
    for a in ulist:
        slist[a]=list()
    a=0
    while a < (len(data1)):
        vector=data1[a]
        value=vector[-1]
        slist[value].append(vector)
        a+=1
    return slist

In [40]:
def summarize(data1):
    summaries = [(mean(column), std_dev(column), len(column)) for column in zip(*data1)]
    del(summaries[-1])
    return summaries

In [41]:
def predict(summaries, row):
    prob = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for value, probability in prob.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = value
    return best_label

In [42]:
def calculate_probability(x, mean, stddev):
    if stddev==0.0 or mean==0.0:
        return 0.0
    exponent = exp(-((x-mean)**2 / (2 * stddev**2 )))
    return (1 / (sqrt(2 * pi) * stddev)) * exponent

In [43]:
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    prob = dict()
    for value, class_summaries in summaries.items():
        prob[value] = summaries[value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stddev, _ = class_summaries[i]
            prob[value] *= calculate_probability(row[i], mean, stddev)
    return prob

In [44]:
def mean(num):
    return sum(num)/float(len(num))
def std_dev(num):
    avg=mean(num)
    var = sum([(x-avg)**2 for x in num]) / float(len(num)-1) 
    return sqrt(var)

In [45]:
def acc_metric(act, pred):
    crct = 0
    a=0
    while a<(len(act)):
        if act[a] == pred[a]:
            crct += 1
        a+=1
    return crct / float(len(act)) * 100.0

In [46]:
scores=eval_algo(data1,10)
print('Scores: %s' % scores)
print('Mean Accuracy: %.4f%%' % (sum(scores)/float(len(scores))))

[array([597, 0, 0, 2, 0, 1, 1, 2], dtype=object), array([449, 0, 3, 0, 1, 0, 0, 2], dtype=object), array([886, 2, 3, 0, 2, 1, 2, 2], dtype=object), array([279, 3, 2, 2, 1, 2, 1, 2], dtype=object), array([1543, 1, 2, 1, 0, 1, 2, 2], dtype=object), array([1255, 2, 1, 2, 1, 1, 2, 1], dtype=object), array([1092, 2, 2, 0, 1, 1, 1, 2], dtype=object), array([1215, 2, 1, 1, 0, 2, 1, 2], dtype=object), array([1423, 1, 0, 0, 2, 2, 2, 2], dtype=object), array([1288, 2, 1, 3, 2, 2, 2, 0], dtype=object), array([933, 2, 3, 2, 1, 0, 1, 2], dtype=object), array([1035, 2, 0, 2, 1, 2, 1, 2], dtype=object), array([1129, 2, 2, 1, 2, 1, 2, 0], dtype=object), array([1153, 2, 2, 2, 2, 2, 2, 0], dtype=object), array([1375, 1, 3, 2, 2, 0, 2, 0], dtype=object), array([1037, 2, 0, 2, 1, 2, 0, 0], dtype=object), array([1549, 1, 2, 1, 1, 2, 2, 0], dtype=object), array([1601, 1, 2, 3, 0, 0, 0, 2], dtype=object), array([4, 3, 3, 0, 0, 1, 2, 2], dtype=object), array([959, 2, 3, 3, 1, 1, 0, 0], dtype=object), array([1